In [1]:
import json
import pandas as pd
import os
import numpy as np
import glob
from konlpy.tag import Mecab

In [4]:
# 원본 데이터 개수 확인
original_ds =  "/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/raw_data/feed_data_20210630_2249(fixed_abnormal_extension)(전체_0~281357)(FACES, REALIMG).xlsx"

original_df = pd.read_excel(original_ds)['INDEX']
len(original_df)

281358

In [5]:
original_df.head()

0    0
1    1
2    2
3    3
4    4
Name: INDEX, dtype: int64

In [ ]:
# 비식별화 결과 파일 확인
def number_deidentified_files(filepath, original_data):
    deidentified_df = pd.DataFrame(columns=["INDEX"])
    
    for file in os.listdir(filepath):
        try:
            data = pd.read_excel(os.path.join(filepath, file), encoding="utf-8")
            data = data[["INDEX"]] #Count by index(No duplicated data!)
            deidentified_df = pd.concat([deidentified_df, data], ignore_index=True)
            deidentified_df = deidentified_df.sort_values(by=['INDEX'])
        except:
            pass
    deidentified_df = deidentified_df.reset_index(drop=True)
    print(f'Number of original datas: {len(original_data)}')
    print(f'Number of deidentified datas: {len(deidentified_df)}')
    print(f'Number of duplicated index of original datas: {len(original_data[original_data.duplicated()==True])}')    
    print(f'Number of duplicated index of deidentified datas: {len(deidentified_df[deidentified_df.duplicated()==True])}')
    
    return len(original_data) - len(deidentified_df)

In [ ]:
number_deidentified_files(deidentification_path, original_df)

In [ ]:
# invalid 여부가 y인 데이터 개수 확인하기
len(deidentification_df[deidentification_df['FACES']=='y'])

In [ ]:
deidentification_df[deidentification_df['FACES']=='y'].head(30)

In [11]:
#json파일의 개수 확인

def number_json_files(directories, original_datas):
        index_image = []
        number_of_texts = 0
        
    
        for directory in directories:
            for jsonfile in os.listdir(directory):
                with open(os.path.join(directory, jsonfile), "r", encoding="utf-8") as f:
                    f = json.load(f)
                    image_list = f["image_label"]
                    text = f["text_label"]

                    for image in image_list:
                        index_image.append(image["index"])
            number_of_texts += len(os.listdir(directory))

                    
        number_of_images = len(index_image) 
        
        number_of_datas = {
            'images':number_of_images,
            'texts':number_of_texts
        }
        
        print(f'Number of datas: {number_of_datas}')
        
        remains = {
            'remain_images': original_datas['images'] - number_of_datas['images'],
            'remain_texts':  original_datas['texts'] - number_of_datas['texts']
        }
        
        return f'Need more datas: {remains}'

In [26]:
path = "/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset"
original_ds = os.path.join(path, "circlin_feeds_dataset/raw_data/feed_data_20210630_2249(fixed_abnormal_extension)(전체_0~281357)(FACES, REALIMG).xlsx")
deidentification_path = os.path.join(path, "deidentification/deidentification_completed_20211029.xlsx")
json_dirs = [
    os.path.join(path, '1_210820(5375)'),
    os.path.join(path, '2_210902(78763)'),
    os.path.join(path, '3_210924(46060)'),
    os.path.join(path, '4_211012(37112)'),
    os.path.join(path, '5_211029(10749)'),
    os.path.join(path, '5_211029_handwork(10)'),
    os.path.join(path, '7_211112(28284)'),
    os.path.join(path, '8_211112(35074)'),
    os.path.join(path, '9_211117(1028)')
]

number_original_data = {
    'images': 281358,
    'texts': 194389
}

In [27]:
#Find error images

all_none_index = []
invalidImage_index = []
labeled_index = []
for directory in json_dirs:
    for jsonfile in os.listdir(directory):
        with open(os.path.join(path, directory, jsonfile), "r", encoding="utf-8") as f:
            f = json.load(f)
            image_list = f["image_label"]
            
            for image in image_list:
                if image["athletics"][0] == "none" and image["foods"][0] == "none" and image["etc"][0] == "none":
                    try:
                        if image["invalidImage"]==True or image["invalidImage"]=="보이지 않음":
                            print(f"{directory} 디렉터리, 에러 이미지:{f['sourceData']['seq'], image['index'], image['invalidImage']}")
                            invalidImage_index.append(image['index'])
                    except:
                        print(f"{directory} 디렉터리, {f['sourceData']['seq'], image['index']}")
                        all_none_index.append(image['index'])
                else:
                    labeled_index.append(image["index"])

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (7304, 6908)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (5596, 5235)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (5295, 4942)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (271, 71)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (1152, 915)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (323, 119)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (1869, 1619)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (3866, 3548)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (1809, 1561)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/1_210820(5375) 디렉터리, (1887, 1637)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_2109

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(364522, 236919, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(357318, 227542, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(309197, 175072, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(359847, 230845, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(310254, 176478, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(310254, 176479, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(293573, 169570, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(296264, 173046, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(787

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(324180, 192570, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(317107, 185211, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(362589, 234439, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(362589, 234441, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(317998, 186137, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(316241, 184113, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(316241, 184114, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(291678, 167061, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(787

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(313161, 180163, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, (276954, 150650)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, (276954, 150651)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(324111, 192497, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(314579, 182001, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(311797, 178515, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(314835, 182333, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(312930, 179841, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(356675, 226694, 

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(363991, 236180, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(363991, 236182, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(358189, 228692, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(297367, 174537, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(310295, 176533, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(316306, 184181, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(296897, 173878, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(78763) 디렉터리, 에러 이미지:(296897, 173880, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/2_210902(787

...

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(146419, 91051, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(146419, 91052, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(146419, 91053, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, (370643, 244649)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(140832, 88775, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(140832, 88776, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(381004, 256565, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(129168, 84607, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(12916

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, (105751, 74824)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(105751, 74826, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(369044, 242682, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(369044, 242683, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, (369727, 243546)
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(366780, 239764, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(384084, 260441, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(374944, 249716, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(152820, 93564, '보이

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(367112, 240205, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(368396, 241806, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(121158, 81576, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(121158, 81577, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(380529, 255964, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(365798, 238551, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(375829, 250686, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(385542, 262337, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060

/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(142633, 89410, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(137339, 87537, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(149188, 92161, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(149188, 92162, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(149188, 92163, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(371343, 245491, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(371343, 245492, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉터리, 에러 이미지:(149184, 92153, '보이지 않음')
/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/3_210924(46060) 디렉

KeyboardInterrupt: 

In [ ]:
invalidImage_index[0:30]

In [ ]:
print(len(all_none_index), len(invalidImage_index), len(labeled_index))
print(len(all_none_index) + len(invalidImage_index) + len(labeled_index))

In [28]:
print(len(deidentification_df[deidentification_df['FACES']=='n'])) #209652
print(len(deidentification_df[deidentification_df['FACES']=='y'])) #56139
print(len(deidentification_df[deidentification_df['FACES']=='Y'])) #17
print(len(deidentification_df[deidentification_df['FACES']=='y '])) #1
print(len(deidentification_df[deidentification_df['FACES']=='x'])) #527
print(len(deidentification_df[deidentification_df['FACES']=='X'])) #22

222806
58003
0
0
549
0


In [29]:
#라벨별 빈도 체크
def find_labels(directories): #list
    dict_image_label = {}
    dict_text_label = {}
    num_images = 0
    for directory in directories:
        for jsonfile in os.listdir(directory):
            with open(os.path.join(directory, jsonfile), "r", encoding="utf-8") as f:
                f = json.load(f)
                image_list = f["image_label"]
                text = f["text_label"]
                
                #Image label #Consider invalidImage
                for image in image_list:
                    if image["athletics"][0] != "none":
                        for athletics_label in image["athletics"]:
                            if athletics_label in dict_image_label.keys():
                                dict_image_label[athletics_label] += 1
                            else:
                                print(f"Key {athletics_label} first added: {f['sourceData']['seq']}")
                                dict_image_label[athletics_label] = 0
                    else:
                        pass
                    if image["foods"][0] != "none":
                        for foods_label in image["foods"]:
                            if foods_label in dict_image_label.keys():
                                dict_image_label[foods_label] += 1
                            else:
                                print(f"Key {foods_label} first added: {f['sourceData']['seq']}")                                
                                dict_image_label[foods_label] = 0
                    else:
                        pass
                    if image["etc"][0] != "none":
                        for etc_label in image["etc"]:
                            if etc_label in dict_image_label.keys():
                                dict_image_label[etc_label] += 1
                            else:
                                print(f"Key {etc_label} first added: {f['sourceData']['seq']}")                                    
                                dict_image_label[etc_label] = 0
                    else:
                        pass                   
                
                #Text label
                if text["athletics"][0] != "none":
                    for athletics_label in text["athletics"]:
                        if athletics_label in dict_text_label.keys():
                            dict_text_label[athletics_label] += 1
                        else:
                            dict_text_label[athletics_label] = 0
                else:
                    pass
                if text["foods"][0] != "none":
                    for foods_label in text["foods"]:
                        if foods_label in dict_text_label.keys():
                            dict_text_label[foods_label] += 1
                        else:
                            dict_text_label[foods_label] = 0
                else:
                    pass
                if text["etc"][0] != "none":
                    for etc_label in text["etc"]:
                        if etc_label in dict_text_label.keys():
                            dict_text_label[etc_label] += 1
                        else:
                            dict_text_label[etc_label] = 0
                else:
                    pass
    return [dict_image_label, dict_text_label]

In [30]:
labels_dict = find_labels(json_dirs)

Key 기타운동 first added: 8592
Key 신체기록 first added: 8592
Key 웨이트 first added: 8472
Key 루틴기록 first added: 8472
Key 일상생활 first added: 5050
Key 유산소기록 first added: 5621
Key 운동기구 first added: 6341
Key 스키/스노보드 first added: 3661
Key 홈트 first added: 312
Key 의류 first added: 312
Key 종합운동 first added: 2463
Key 요가 first added: 3060
Key 맨몸 first added: 2804
Key 달리기/조깅 first added: 6046
Key 무술 first added: 3251
Key 수상스포츠 first added: 895
Key 운동용품 first added: 895
Key 등산/등반 first added: 573
Key 격투기 first added: 7197
Key 보충제 first added: 4254
Key 자전거 first added: 3285
Key 필라테스 first added: 7643
Key 온라인클래스 first added: 5021
Key 걷기/산책 first added: 3742
Key 일반식 first added: 22
Key 건강간식 first added: 3448
Key 건강음료 first added: 7284
Key 건강식 first added: 6522
Key 기타식단 first added: 833
Key 폴댄스 first added: 6385
Key 일반음료 first added: 2429
Key 농구 first added: 3258
Key 줄넘기 first added: 4626
Key 축구/풋살 first added: 5097
Key 보조제 first added: 8383
Key 골프 first added: 9273
Key 테니스 first added: 8613
Key 간편식 first added: 

In [31]:
#Image label
sorted(labels_dict[0].items(), key=(lambda x: x[1]), reverse=True)

[('유산소기록', 71672),
 ('기타운동', 53936),
 ('달리기/조깅', 50206),
 ('웨이트', 26098),
 ('운동기구', 22293),
 ('건강식', 21925),
 ('걷기/산책', 19056),
 ('일반식', 12252),
 ('신체기록', 12136),
 ('맨몸', 11172),
 ('자전거', 10723),
 ('홈트', 10474),
 ('일상생활', 10153),
 ('운동용품', 9095),
 ('건강음료', 9029),
 ('온라인클래스', 6715),
 ('등산/등반', 3749),
 ('의류', 3395),
 ('루틴기록', 2967),
 ('기타식단', 2958),
 ('일반음료', 2702),
 ('종합운동', 1711),
 ('일반간식', 1672),
 ('건강간식', 1387),
 ('보조제', 1300),
 ('요가', 1237),
 ('보충제', 1156),
 ('수상스포츠', 1044),
 ('필라테스', 925),
 ('간편식', 755),
 ('격투기', 372),
 ('골프', 354),
 ('축구/풋살', 320),
 ('줄넘기', 232),
 ('테니스', 223),
 ('무술', 208),
 ('농구', 71),
 ('폴댄스', 70),
 ('배드민턴', 61),
 ('볼링', 55),
 ('야구', 44),
 ('승마', 41),
 ('탁구', 28),
 ('스키/스노보드', 26),
 ('배구', 3),
 ('당구', 2)]

In [32]:
#Text label
sorted(labels_dict[1].items(), key=(lambda x: x[1]), reverse=True)

[('웨이트', 85120),
 ('달리기/조깅', 57864),
 ('기타운동', 51002),
 ('일상생활', 31551),
 ('맨몸', 25270),
 ('걷기/산책', 24846),
 ('유산소기록', 22188),
 ('건강식', 15615),
 ('자전거', 14979),
 ('루틴기록', 10824),
 ('기타식단', 9964),
 ('일반식', 9921),
 ('홈트', 9130),
 ('건강음료', 8857),
 ('온라인클래스', 7127),
 ('등산/등반', 6815),
 ('필라테스', 3811),
 ('종합운동', 2968),
 ('요가', 2843),
 ('신체기록', 2020),
 ('수상스포츠', 1915),
 ('일반음료', 1694),
 ('일반간식', 1465),
 ('운동기구', 1358),
 ('건강간식', 1262),
 ('보충제', 1259),
 ('무술', 1222),
 ('보조제', 998),
 ('의류', 728),
 ('격투기', 689),
 ('운동용품', 689),
 ('축구/풋살', 625),
 ('골프', 528),
 ('줄넘기', 451),
 ('테니스', 307),
 ('폴댄스', 299),
 ('간편식', 290),
 ('배드민턴', 157),
 ('농구', 105),
 ('승마', 101),
 ('볼링', 86),
 ('야구', 85),
 ('탁구', 51),
 ('스키/스노보드', 43),
 ('배구', 10),
 ('당구', 4)]